# Network Security Capstone - Data Overview

**Purpose:** Load and explore both BETH and UNSW-NB15 datasets to understand their structure, characteristics, and suitability for anomaly detection and classification tasks.

---

## Datasets:
- **BETH Dataset**: Honeypot system call logs for unsupervised anomaly detection
- **UNSW-NB15 Dataset**: Network traffic data for supervised attack classification

---

**Author:** Joshua Laubach  
**Date:** October 27, 2025

## Project Setup

**Purpose:** This section outlines the necessary steps to configure the environment for this notebook.

### Prerequisites
*   **Python Environment:** Python 3.8+ is recommended.
*   **Dependencies:** All required packages are listed in `requirements.txt` in the project's root directory. Install them before proceeding:
    ```bash
    pip install -r ../../requirements.txt
    ```

### Project Structure
This notebook is located in the `notebooks/` directory and depends on custom modules in the `src/` directory. The code automatically adds `src/` to the system path for seamless imports. The expected structure is:
```
network_security_capstone/
|-- notebooks/
|   `-- 01_data_overview.ipynb
|-- src/
|   `-- ... (custom modules)
`-- requirements.txt
```
---


## Executive Summary

This exploratory data analysis examines two cybersecurity datasets for anomaly detection and attack classification modeling:

### BETH Dataset - Key Findings
- **Purpose**: Unsupervised anomaly detection on honeypot system call logs
- **Class Imbalance**: Training set is ~99% normal (ideal for unsupervised learning)
- **Data Quality**: No missing values, but high-cardinality text features require preprocessing
- **Preprocessing Impact**: 10-15% memory reduction, log transforms handle skewed distributions
- **Modeling Challenge**: Extremely rare anomalies in training data (~1% suspicious/evil combined)

### UNSW-NB15 Dataset - Key Findings
- **Purpose**: Supervised classification of network attacks (binary + multi-class)
- **Class Distribution**: ~54% normal, ~46% attack (relatively balanced for supervised learning)
- **Attack Diversity**: 9 distinct attack types (Fuzzers, DoS, Exploits, Reconnaissance, etc.)
- **Missing Data**: Some null values present in raw data, handled via dropna in preprocessing
- **Preprocessing Impact**: Categorical encoding enables ML, validation split created for tuning

### Data Readiness Assessment
- **Both datasets preprocessed and scaled** (Z-score normalization, mean~=0, std~=1)
- **Outliers persist after preprocessing** but at manageable levels (z-score method shows 1-5% outliers)
- **Feature engineering applied**: Log transforms, binary flags, categorical encoding
- **Memory optimized**: 5-15% reduction through type optimization and column removal
- **Validation splits available** for proper model evaluation

### Recommended Modeling Approaches

**BETH (Unsupervised Anomaly Detection):**
- Autoencoder, Isolation Forest, One-Class SVM
- Train on normal data only (training set)
- Evaluate on test set with labeled anomalies

**UNSW-NB15 (Supervised Classification):**
- Binary: RandomForest, XGBoost, Neural Networks
- Multi-class: Gradient Boosting, Deep Learning for attack type identification
- Use validation set for hyperparameter tuning

---

## Table of Contents

1. [Import Required Libraries](#1-import-required-libraries)
2. [Raw Data Analysis](#2-raw-data-analysis)
   - 2.1 Load Raw Datasets
   - 2.2 BETH Raw Data Overview
   - 2.3 UNSW-NB15 Raw Data Overview
   - 2.4 Target Distribution Across Splits
   - 2.5 Feature Correlation Analysis
   - 2.6 Note on Outlier Detection
3. [Preprocessed Data Analysis](#3-preprocessed-data-analysis)
   - 3.1 Load Preprocessed Datasets
   - 3.2 BETH Preprocessed Data Overview
   - 3.3 UNSW-NB15 Preprocessed Data Overview
   - 3.4 Outlier Detection on Preprocessed Data
4. [Data Comparison & Insights](#4-data-comparison--insights)
   - 4.1 Before vs After Preprocessing
   - 4.2 Key Insights & Preprocessing Impact
5. [Summary](#5-summary)

---

## 1. Import Required Libraries

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')

# Add the src directory to the Python path to allow for importing custom modules
src_path = os.path.abspath(os.path.join('..', 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)

# Now you can import your custom modules
from data_loading import load_beth, load_unsw
from data_extraction import extract_beth, extract_unsw
from preprocessing import preprocess_beth_split, preprocess_unsw_split

# Set plot style
sns.set(style="whitegrid")

# 2. Raw Data Analysis

**Purpose**: Explore the **original unprocessed datasets** to understand raw data characteristics, distributions, and quality issues.

This section analyzes data **before** any preprocessing, feature engineering, or transformations are applied.

---

## 2.1 Load Raw Datasets

In [ ]:
# Load raw unprocessed datasets for EDA
beth_train_raw, beth_val_raw, beth_test_raw = extract_beth(save_to_disk=True)
unsw_train_raw, unsw_test_raw = extract_unsw(save_to_disk=True)

print("Raw datasets loaded successfully")

## 2.2 BETH Raw Data Overview

**BETH (BPF-Extended Telemetry for Honeypots)** - Original system call logs from honeypot systems.

### Key Characteristics:
- **Raw Timestamps**: Original timestamp values (not renamed or transformed)
- **Text Columns**: JSON `args` column intact
- **All Features**: processName, threadId, stackAddresses, eventName present
- **No Encoding**: hostName as string, not encoded
- **Targets**: `sus` (suspicious) and `evil` (malicious) labels

In [ ]:
# BETH RAW Dataset Analysis
print("BETH DATASET - RAW DATA CHARACTERISTICS")
print("=" * 80)

print(f"\nDataset Sizes:")
print(f"  Training:   {beth_train_raw.shape}")
print(f"  Validation: {beth_val_raw.shape}")
print(f"  Test:       {beth_test_raw.shape}")

print(f"\nMemory Usage:")
print(f"  Training:   {beth_train_raw.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"  Validation: {beth_val_raw.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"  Test:       {beth_test_raw.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print(f"\nColumn Names ({len(beth_train_raw.columns)} total):")
if len(beth_train_raw.columns) <= 10:
    print(f"  {list(beth_train_raw.columns)}")
else:
    print(f"  {list(beth_train_raw.columns[:10])} ... and {len(beth_train_raw.columns) - 10} more")

print(f"\nData Types:")
print(beth_train_raw.dtypes.value_counts())

print(f"\nSample Rows:")
display(beth_train_raw.head(3).T)

print(f"\nNull Values:")
null_counts = beth_train_raw.isnull().sum()
if null_counts.sum() > 0:
    print(null_counts[null_counts > 0])
else:
    print("  No null values")

print(f"\nTarget Distribution:")
print(f"  Train Set (used for unsupervised learning):")
if 'sus' in beth_train_raw.columns and 'evil' in beth_train_raw.columns:
    normal_train = ((beth_train_raw['sus']==0) & (beth_train_raw['evil']==0)).sum()
    sus_train = (beth_train_raw['sus']==1).sum()
    evil_train = (beth_train_raw['evil']==1).sum()
    total_train = len(beth_train_raw)
    print(f"    Normal: {normal_train:,} ({100*normal_train/total_train:.2f}%)")
    print(f"    Suspicious: {sus_train:,} ({100*sus_train/total_train:.2f}%)")
    print(f"    Evil: {evil_train:,} ({100*evil_train/total_train:.2f}%)")

print(f"\n  Test Set (contains all anomalies for evaluation):")
if 'sus' in beth_test_raw.columns and 'evil' in beth_test_raw.columns:
    normal_test = ((beth_test_raw['sus']==0) & (beth_test_raw['evil']==0)).sum()
    sus_test = (beth_test_raw['sus']==1).sum()
    evil_test = (beth_test_raw['evil']==1).sum()
    total_test = len(beth_test_raw)
    print(f"    Normal: {normal_test:,} ({100*normal_test/total_test:.2f}%)")
    print(f"    Suspicious: {sus_test:,} ({100*sus_test/total_test:.2f}%)")
    print(f"    Evil: {evil_test:,} ({100*evil_test/total_test:.2f}%)")

print(f"\nNote: BETH has 2 binary target variables:")
print(f"  - 'sus' (suspicious): anomalous behavior")
print(f"  - 'evil' (malicious): confirmed malicious activity")
print(f"  - Train/Val sets are ~99% normal for unsupervised anomaly detection training")
print(f"  - Test set contains all anomalous samples for model evaluation")

print(f"\nNumeric vs Categorical Features:")
numeric_raw = beth_train_raw.select_dtypes(include=['float64', 'int64']).columns
categorical_raw = beth_train_raw.select_dtypes(include=['object']).columns
print(f"  Numeric: {len(numeric_raw)} columns")
print(f"  Categorical: {len(categorical_raw)} columns")
print(f"  Total: {len(beth_train_raw.columns)} columns")

## 2.3 UNSW-NB15 Raw Data Overview

**UNSW-NB15** - Original network traffic dataset created by UNSW Canberra Cyber Range Lab.

### Key Characteristics:
- **Categorical Features**: proto, service, state as strings (not encoded)
- **All 49 Original Features**: No dropped columns
- **ID Column**: Present (row identifier)
- **Missing Values**: May be present (not cleaned)
- **Targets**: `label` (binary: 0=normal, 1=attack) and `attack_cat` (multi-class)

In [ ]:
# UNSW-NB15 RAW Dataset Analysis
print("UNSW-NB15 DATASET - RAW DATA CHARACTERISTICS")
print("=" * 80)

print(f"\nDataset Sizes:")
print(f"  Training: {unsw_train_raw.shape}")
print(f"  Test:     {unsw_test_raw.shape}")

print(f"\nMemory Usage:")
print(f"  Training: {unsw_train_raw.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"  Test:     {unsw_test_raw.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print(f"\nColumn Names ({len(unsw_train_raw.columns)} total):")
if len(unsw_train_raw.columns) <= 10:
    print(f"  {list(unsw_train_raw.columns)}")
else:
    print(f"  {list(unsw_train_raw.columns[:10])} ... and {len(unsw_train_raw.columns) - 10} more")

print(f"\nData Types:")
print(unsw_train_raw.dtypes.value_counts())

print(f"\nSample Rows:")
display(unsw_train_raw.head(3).T)

print(f"\nNull Values:")
null_counts = unsw_train_raw.isnull().sum()
if null_counts.sum() > 0:
    print(null_counts[null_counts > 0])
else:
    print("  No null values")

print(f"\nTarget Distribution (Train Set):")
if 'label' in unsw_train_raw.columns:
    normal = (unsw_train_raw['label'] == 0).sum()
    attack = (unsw_train_raw['label'] == 1).sum()
    total = len(unsw_train_raw)
    print(f"  Binary Label ('label'):")
    print(f"    Normal (0): {normal:,} ({100*normal/total:.2f}%)")
    print(f"    Attack (1): {attack:,} ({100*attack/total:.2f}%)")

print(f"\nNote: UNSW-NB15 has 2 target variables:")
print(f"  - 'label': Binary (0=Normal, 1=Attack) - derived as: 0 if attack_cat=='Normal' else 1")
print(f"  - 'attack_cat': Multi-class with 10 categories (1 Normal + 9 attack types)")

print(f"\nTop 10 Attack Categories ('attack_cat'):")
for idx, cat in enumerate(unsw_train_raw['attack_cat'].value_counts().head(10).index):
    count = (unsw_train_raw['attack_cat'] == cat).sum()
    is_normal = "<- Normal class" if cat == 'Normal' else "<- Attack type"
    print(f"  {idx+1}. {cat:20s} : {count:8,} samples {is_normal}")

print(f"\nNumeric vs Categorical Features:")
numeric_unsw_raw = unsw_train_raw.select_dtypes(include=['float64', 'int64']).columns
categorical_unsw_raw = unsw_train_raw.select_dtypes(include=['object']).columns
print(f"  Numeric: {len(numeric_unsw_raw)} columns")
print(f"  Categorical: {len(categorical_unsw_raw)} columns")
print(f"  Total: {len(unsw_train_raw.columns)} columns")

# Define numeric features for UNSW-NB15
unsw_numeric = unsw_train_raw[numeric_unsw_raw]

In [ ]:
# Visualize UNSW-NB15 Raw Data Distributions
fig, axs = plt.subplots(2, 3, figsize=(18, 10))
axs = axs.flatten()
fig.suptitle('UNSW-NB15 Raw Data - Feature Distributions', fontsize=16, fontweight='bold')

# Select 6 numeric features with highest variance
unsw_numeric_features = [col for col in numeric_unsw_raw if col not in ['label', 'id']]
variances_unsw = unsw_train_raw[unsw_numeric_features].var().sort_values(ascending=False)
top_6_unsw = variances_unsw.head(6).index.tolist()

for idx, feature in enumerate(top_6_unsw):
    # Histogram
    axs[idx].hist(unsw_train_raw[feature].dropna(), bins=50, alpha=0.7, 
            color='coral', edgecolor='black')
    axs[idx].set_xlabel(feature, fontsize=11, fontweight='bold')
    axs[idx].set_ylabel('Frequency', fontsize=11)
    axs[idx].set_title(f'Distribution: {feature}', fontsize=12, fontweight='bold')
    axs[idx].grid(alpha=0.3)
    
    # Add statistics
    mean_val = unsw_train_raw[feature].mean()
    median_val = unsw_train_raw[feature].median()
    axs[idx].axvline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_val:.2e}')
    axs[idx].axvline(median_val, color='green', linestyle='--', linewidth=2, label=f'Median: {median_val:.2e}')
    axs[idx].legend(fontsize=9)

plt.tight_layout()
plt.show()

print(f"Displayed raw distributions for top 6 features by variance")

In [ ]:
# Visualize Source-Destination Relationships
fig, axs = plt.subplots(3, 3, figsize=(18, 16))
axs = axs.flatten()
fig.suptitle('UNSW-NB15: Source-Destination Patterns by Attack Label', 
             fontsize=16, fontweight='bold')

# Sample data for visualization
sample_size = len(unsw_train_raw)
sample_indices = np.random.choice(len(unsw_train_raw), sample_size, replace=False)
sample_data = unsw_train_raw.iloc[sample_indices]

# Define scatter plot pairs
scatter_pairs = [
    ('sbytes', 'dbytes', 'Source Bytes vs Destination Bytes'),
    ('spkts', 'dpkts', 'Source Packets vs Destination Packets'),
    ('sload', 'dload', 'Source Load vs Destination Load'),
    ('sloss', 'dloss', 'Source Loss vs Destination Loss'),
    ('sinpkt', 'dinpkt', 'Source Inter-Packet Time vs Dest IPT'),
    ('sjit', 'djit', 'Source Jitter vs Destination Jitter'),
    ('smean', 'dmean', 'Source Mean Packet Size vs Dest Mean'),
    ('swin', 'dwin', 'Source TCP Window vs Dest TCP Window')
]

# Create scatter plots
for idx, (x_col, y_col, title) in enumerate(scatter_pairs):
    if x_col not in sample_data.columns or y_col not in sample_data.columns:
        axs[idx].text(0.5, 0.5, f'Columns {x_col}/{y_col} not found', 
                     ha='center', va='center', fontsize=12)
        axs[idx].set_title(title, fontsize=11, fontweight='bold')
        continue
    
    normal_mask = sample_data['label'] == 0
    attack_mask = sample_data['label'] == 1
    
    # Plot normal traffic
    axs[idx].scatter(sample_data.loc[normal_mask, x_col], 
                    sample_data.loc[normal_mask, y_col],
                    alpha=0.8, s=15, c='steelblue', label='Normal', edgecolors='none')
    
    # Plot attack traffic
    axs[idx].scatter(sample_data.loc[attack_mask, x_col], 
                    sample_data.loc[attack_mask, y_col],
                    alpha=0.3, s=15, c='coral', label='Attack', edgecolors='none')
    
    axs[idx].set_xlabel(x_col, fontsize=10, fontweight='bold')
    axs[idx].set_ylabel(y_col, fontsize=10, fontweight='bold')
    axs[idx].set_title(title, fontsize=11, fontweight='bold')
    axs[idx].legend(loc='upper right', fontsize=8)
    axs[idx].grid(True, alpha=0.3)
    
    # Add diagonal reference line
    x_max = sample_data[x_col].max()
    y_max = sample_data[y_col].max()
    if x_max > 0 and y_max > 0:
        max_val = max(x_max, y_max)
        axs[idx].plot([0, max_val], [0, max_val], 'k--', alpha=0.3, linewidth=1)

axs[8].axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Visualize Log-Transformed Source-Destination Relationships
fig, axs = plt.subplots(3, 3, figsize=(18, 16))
axs = axs.flatten()
fig.suptitle('UNSW-NB15: Log-Transformed Source-Destination Patterns', 
             fontsize=16, fontweight='bold')

sample_data_log = sample_data.copy()

for idx, (x_col, y_col, title) in enumerate(scatter_pairs):
    if x_col not in sample_data_log.columns or y_col not in sample_data_log.columns:
        axs[idx].text(0.5, 0.5, f'Columns {x_col}/{y_col} not found', 
                     ha='center', va='center', fontsize=12)
        axs[idx].set_title(f'Log: {title}', fontsize=11, fontweight='bold')
        continue
    
    # Apply log1p transformation
    log_x = np.log1p(sample_data_log[x_col])
    log_y = np.log1p(sample_data_log[y_col])
    
    normal_mask = sample_data_log['label'] == 0
    attack_mask = sample_data_log['label'] == 1
    
    axs[idx].scatter(log_x[normal_mask], log_y[normal_mask],
                    alpha=0.8, s=15, c='steelblue', label='Normal', edgecolors='none')
    axs[idx].scatter(log_x[attack_mask], log_y[attack_mask],
                    alpha=0.3, s=15, c='coral', label='Attack', edgecolors='none')

    axs[idx].set_xlabel(f'$\\log(1 + {x_col})$', fontsize=10, fontweight='bold')
    axs[idx].set_ylabel(f'$\\log(1 + {y_col})$', fontsize=10, fontweight='bold')
    axs[idx].set_title(f'Log: {title}', fontsize=11, fontweight='bold')
    axs[idx].legend(loc='upper right', fontsize=8)
    axs[idx].grid(True, alpha=0.3)
    
    # Add diagonal reference line
    x_max = log_x.max()
    y_max = log_y.max()
    if x_max > 0 and y_max > 0:
        max_val = max(x_max, y_max)
        axs[idx].plot([0, max_val], [0, max_val], 'k--', alpha=0.3, linewidth=1)

axs[8].axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# UNSW-NB15 Target Distribution (Binary + Multi-class)
print("\n=== UNSW-NB15 Target Variables ===")
print("\nBinary Target ('label' - derived from 'attack_cat'):")
print(unsw_train_raw['label'].value_counts().sort_index())
print(f"Attack ratio: {unsw_train_raw['label'].mean():.2%}")

print("\nMulti-class Target ('attack_cat' - primary classification target):")
attack_counts = unsw_train_raw['attack_cat'].value_counts()
for cat, count in attack_counts.items():
    pct = count / len(unsw_train_raw) * 100
    cat_type = "Normal class" if cat == 'Normal' else "Attack type"
    print(f"  {cat:20s}: {count:8,} ({pct:5.2f}%)  [{cat_type}]")

print("\nMODELING STRATEGIES:")
print("  1. Binary Classification:")
print("     -> Use 'label' as target")
print("     -> Simpler problem, higher accuracy expected")
print()
print("  2. Multi-class Classification:")
print("     -> Use 'attack_cat' as target")
print("     -> More challenging, provides attack-specific insights")
print("     -> Can identify specific attack types (Fuzzers, DoS, Exploits, etc.)")

# Target Distribution Comparison
fig, axs = plt.subplots(1, 2, figsize=(14, 5))
axs = axs.flatten()

# BETH target distribution
axs[0].pie([beth_train_raw['sus'].sum(), len(beth_train_raw) - beth_train_raw['sus'].sum()],
           labels=['Suspicious', 'Normal'], autopct='%1.1f%%', startangle=90)
axs[0].set_title('BETH: Suspicious vs Normal (Train Set)')

# UNSW target distribution
axs[1].pie([unsw_train_raw['label'].sum(), len(unsw_train_raw) - unsw_train_raw['label'].sum()],
           labels=['Attack', 'Normal'], autopct='%1.1f%%', startangle=90)
axs[1].set_title('UNSW-NB15: Attack vs Normal (Train Set)')

plt.tight_layout()
plt.show()

# 3. Preprocessed Data Analysis

**Purpose**: Explore the **preprocessed datasets** after all transformations, feature engineering, and scaling.

This section analyzes data **after** the complete preprocessing pipeline to understand how the data has been transformed for modeling.

---

## 3.1 Load Preprocessed Datasets

In [ ]:
# Load preprocessed datasets for modeling
beth_train, beth_val, beth_test = load_beth(tfidf=False, save_to_disk=True, verbose=False)
unsw_train, unsw_val, unsw_test = load_unsw(split_test=True, save_to_disk=True, verbose=False)

print("Preprocessed datasets loaded successfully")

## 3.2 BETH Preprocessed Data Overview

**After Preprocessing Pipeline:**
- `timestamp` renamed to `timeSinceBoot`
- Log transforms applied (timeSinceBoot, processId, parentProcessId)
- Feature flags created (*_flag columns)
- Text columns dropped (processName, threadId, stackAddresses, eventName)
- `hostName` encoded to integers
- Numeric features scaled (StandardScaler)
- Binary targets (`sus`, `evil`) preserved as {0, 1}

In [ ]:
# BETH PREPROCESSED Dataset Analysis
print("BETH DATASET - PREPROCESSED DATA CHARACTERISTICS")
print("=" * 80)

print(f"\nDataset Sizes:")
print(f"  Training:   {beth_train.shape}")
print(f"  Validation: {beth_val.shape}")
print(f"  Test:       {beth_test.shape}")

print(f"\nColumns:")
print(f"  {list(beth_train.columns)}")

print(f"\nSample Rows:")
print(beth_train.head(3))

print(f"\nBinary Targets:")
print(f"  sus unique values:  {sorted(beth_train['sus'].unique())}")
print(f"  evil unique values: {sorted(beth_train['evil'].unique())}")

print(f"\nLog-Transformed Features:")
log_features = [col for col in beth_train.columns if col.startswith('log_')]
print(f"  {log_features}")

print(f"\nFeature Flags:")
flag_features = [col for col in beth_train.columns if col.endswith('_flag')]
print(f"  {flag_features}")

print(f"\nNumeric Features:")
numeric_preprocessed = beth_train.select_dtypes(include=['float64', 'int64']).columns
numeric_preprocessed = [c for c in numeric_preprocessed if c not in ['sus', 'evil']]
print(f"  Total scaled features: {len(numeric_preprocessed)}")

print(f"\nScaling Verification (mean~0, std~1):")
for col in numeric_preprocessed[:3]:
    mean_val = beth_train[col].mean()
    std_val = beth_train[col].std()
    print(f"  {col:30s}: mean={mean_val:7.4f}, std={std_val:6.4f}")

## 3.3 UNSW-NB15 Preprocessed Data Overview

**After Preprocessing Pipeline:**
- Test set split in half (validation + test)
- Categorical encoding (proto, service, state to integers)
- `id` column dropped
- Missing values removed (dropna)
- Feature engineering applied
- Log transforms applied to skewed features
- Numeric features scaled (fit on train, transform on val/test)
- Binary targets (`label`, `attack_cat`) preserved

In [ ]:
# UNSW-NB15 PREPROCESSED Dataset Analysis
print("UNSW-NB15 DATASET - PREPROCESSED DATA CHARACTERISTICS")
print("=" * 80)

print(f"\nDataset Sizes:")
print(f"  Training:   {unsw_train.shape}")
print(f"  Validation: {unsw_val.shape}")
print(f"  Test:       {unsw_test.shape}")

print(f"\nColumns (first 20):")
print(f"  {list(unsw_train.columns[:20])}")
print(f"  ... and {len(unsw_train.columns) - 20} more columns")

print(f"\nSample Rows:")
print(unsw_train.head(3))

print(f"\nBinary Label:")
print(f"  label unique values: {sorted(unsw_train['label'].unique())}")

print(f"\nTarget Distributions:")
label_dist_proc = unsw_train['label'].value_counts()
print(f"  Normal (0): {label_dist_proc[0]:,} ({100*label_dist_proc[0]/len(unsw_train):.2f}%)")
print(f"  Attack (1): {label_dist_proc[1]:,} ({100*label_dist_proc[1]/len(unsw_train):.2f}%)")

print(f"\nCategorical Encoding:")
if 'proto' in unsw_train.columns:
    print(f"  proto: {unsw_train['proto'].dtype}, {unsw_train['proto'].nunique()} unique values")
if 'service' in unsw_train.columns:
    print(f"  service: {unsw_train['service'].dtype}, {unsw_train['service'].nunique()} unique values")
if 'state' in unsw_train.columns:
    print(f"  state: {unsw_train['state'].dtype}, {unsw_train['state'].nunique()} unique values")

print(f"\nNumeric Features:")
numeric_unsw_proc = unsw_train.select_dtypes(include=['float64', 'int64']).columns
numeric_unsw_proc = [c for c in numeric_unsw_proc if c not in ['label']]
print(f"  Total features: {len(numeric_unsw_proc)}")

print(f"\nScaling Verification (mean~0, std~1):")
for col in numeric_unsw_proc[:5]:
    mean_val = unsw_train[col].mean()
    std_val = unsw_train[col].std()
    print(f"  {col:30s}: mean={mean_val:7.4f}, std={std_val:6.4f}")

## 3.4 Outlier Detection on Preprocessed Data

**Purpose**: Identify outliers in the **transformed feature space** that models will actually see during training.

**Why After Preprocessing?**
- Scaling changes outlier definitions (z-scores meaningful after standardization)
- Log transforms compress extreme values
- Models train on this data, not raw values
- Need to assess if outliers persist after transformations

In [ ]:
# UNSW: Enhanced Outlier Detection on Preprocessed Data
from scipy import stats

# Select top 4 features by variance from preprocessed data
unsw_preproc_numeric = unsw_train.select_dtypes(include=[np.number])
unsw_preproc_features = [col for col in unsw_preproc_numeric.columns if col not in ['label']]
variances_unsw_preproc = unsw_preproc_numeric[unsw_preproc_features].var().sort_values(ascending=False)
unsw_outlier_features_preproc = variances_unsw_preproc.head(4).index.tolist()

# Visualization 1: Violin Plots (shows distribution + outliers)
fig, axs = plt.subplots(2, 2, figsize=(16, 10))
axs = axs.flatten()
fig.suptitle('UNSW-NB15 (Preprocessed): Outlier Detection - Violin Plots', fontsize=16, fontweight='bold')

for idx, feature in enumerate(unsw_outlier_features_preproc):
    data_normal = unsw_train.loc[unsw_train['label'] == 0, feature].dropna()
    data_attack = unsw_train.loc[unsw_train['label'] == 1, feature].dropna()
    
    # Create violin plot for both classes
    parts = axs[idx].violinplot([data_normal, data_attack], 
                                 positions=[1, 2], 
                                 showmeans=True, 
                                 showmedians=True,
                                 widths=0.7)
    
    # Color the violins
    for pc, color in zip(parts['bodies'], ['steelblue', 'coral']):
        pc.set_facecolor(color)
        pc.set_alpha(0.7)
    
    # Calculate outlier percentages using z-score method (appropriate for scaled data)
    z_scores_n = np.abs(stats.zscore(data_normal))
    outliers_n = data_normal[z_scores_n > 3]  # 3 standard deviations
    
    z_scores_a = np.abs(stats.zscore(data_attack))
    outliers_a = data_attack[z_scores_a > 3]
    
    axs[idx].set_xticks([1, 2])
    axs[idx].set_xticklabels(['Normal', 'Attack'])
    axs[idx].set_ylabel('Scaled Value', fontsize=10, fontweight='bold')
    axs[idx].set_title(f'{feature}\nNormal: {len(outliers_n)} ({100*len(outliers_n)/len(data_normal):.1f}%) | '
                      f'Attack: {len(outliers_a)} ({100*len(outliers_a)/len(data_attack):.1f}%)',
                      fontsize=11, fontweight='bold')
    axs[idx].grid(True, alpha=0.3, axis='y')
    
    # Add reference lines for +-3 sigma
    axs[idx].axhline(3, color='red', linestyle='--', alpha=0.5, linewidth=1)
    axs[idx].axhline(-3, color='red', linestyle='--', alpha=0.5, linewidth=1)

plt.tight_layout()
plt.show()

# Visualization 2: Z-Score Distribution
fig, axs = plt.subplots(2, 2, figsize=(16, 10))
axs = axs.flatten()
fig.suptitle('UNSW-NB15 (Preprocessed): Z-Score Distribution ($|z| > 3$ = Outlier)', 
             fontsize=16, fontweight='bold')

for idx, feature in enumerate(unsw_outlier_features_preproc):
    data = unsw_train[feature].dropna()
    
    # Calculate z-scores
    z_scores = stats.zscore(data)
    
    # Separate inliers and outliers
    inliers = z_scores[np.abs(z_scores) <= 3]
    outliers = z_scores[np.abs(z_scores) > 3]
    
    # Plot histogram
    axs[idx].hist(inliers, bins=50, alpha=0.7, color='steelblue', 
                 label=f'Inliers ({len(inliers):,})', edgecolor='black')
    axs[idx].hist(outliers, bins=30, alpha=0.9, color='red', 
                 label=f'Outliers ({len(outliers):,})', edgecolor='black')
    
    # Add +-3 sigma boundary lines
    axs[idx].axvline(-3, color='orange', linestyle='--', linewidth=2, label='3$\\sigma$ Threshold')
    axs[idx].axvline(3, color='orange', linestyle='--', linewidth=2)
    
    # Add +-1 sigma and +-2 sigma reference lines
    axs[idx].axvline(-2, color='green', linestyle=':', linewidth=1, alpha=0.5)
    axs[idx].axvline(2, color='green', linestyle=':', linewidth=1, alpha=0.5)
    axs[idx].axvline(-1, color='gray', linestyle=':', linewidth=1, alpha=0.3)
    axs[idx].axvline(1, color='gray', linestyle=':', linewidth=1, alpha=0.3)
    
    axs[idx].set_xlabel('Z-Score', fontsize=10, fontweight='bold')
    axs[idx].set_ylabel('Frequency', fontsize=10)
    axs[idx].set_title(f'{feature}\nOutliers: {100*len(outliers)/len(z_scores):.2f}%',
                      fontsize=11, fontweight='bold')
    axs[idx].legend(fontsize=8, loc='upper right')
    axs[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Detailed outlier summary
print("\nUNSW-NB15 PREPROCESSED DATA - OUTLIER ANALYSIS (Z-Score Method)")
print("=" * 95)
print(f"{'Feature':<25} {'Total':<10} {'Outliers':<10} {'%':<8} {'Normal Out':<12} {'Attack Out':<12}")
print("=" * 95)

for feature in unsw_outlier_features_preproc:
    data = unsw_train[feature].dropna()
    z_scores = np.abs(stats.zscore(data))
    outlier_mask = z_scores > 3
    
    # Split by label
    normal_mask = unsw_train['label'] == 0
    attack_mask = unsw_train['label'] == 1
    
    normal_outliers = unsw_train.loc[normal_mask & outlier_mask, feature]
    attack_outliers = unsw_train.loc[attack_mask & outlier_mask, feature]
    total_outliers = outlier_mask.sum()
    
    print(f"{feature:<25} {len(data):<10,} {total_outliers:<10,} {100*total_outliers/len(data):<7.2f}% "
          f"{len(normal_outliers):<12,} {len(attack_outliers):<12,}")

print("\n" + "=" * 95)
print("\nKey Insights:")
print("  - Using Z-Score method ($|z| > 3$) - appropriate for scaled/normalized data")
print("  - After preprocessing, outliers should be significantly reduced")
print("  - Compare outlier rates: do attacks have more extreme z-scores?")
print("  - Low outlier percentage indicates successful preprocessing/scaling")


In [ ]:
# BETH: Enhanced Outlier Detection on Preprocessed Data
# Select top 4 features by variance from preprocessed data
beth_preproc_numeric = beth_train.select_dtypes(include=[np.number])
beth_preproc_features = [col for col in beth_preproc_numeric.columns if col not in ['sus', 'evil']]
variances_beth_preproc = beth_preproc_numeric[beth_preproc_features].var().sort_values(ascending=False)
beth_outlier_features_preproc = variances_beth_preproc.head(4).index.tolist()

# Visualization 1: Violin Plots showing distribution by anomaly label
fig, axs = plt.subplots(2, 2, figsize=(16, 10))
axs = axs.flatten()
fig.suptitle('BETH (Preprocessed): Outlier Detection - Violin Plots by Label', fontsize=16, fontweight='bold')

for idx, feature in enumerate(beth_outlier_features_preproc):
    # Split by sus label (0=normal, 1=suspicious)
    data_normal = beth_train.loc[beth_train['sus'] == 0, feature].dropna()
    data_sus = beth_train.loc[beth_train['sus'] == 1, feature].dropna()
    
    # Create violin plot
    parts = axs[idx].violinplot([data_normal, data_sus], 
                                 positions=[1, 2], 
                                 showmeans=True, 
                                 showmedians=True,
                                 widths=0.7)
    
    # Color the violins
    for pc, color in zip(parts['bodies'], ['steelblue', 'coral']):
        pc.set_facecolor(color)
        pc.set_alpha(0.7)
    
    # Calculate outlier percentages using z-score method
    z_scores_n = np.abs(stats.zscore(data_normal))
    outliers_n = data_normal[z_scores_n > 3]
    
    z_scores_s = np.abs(stats.zscore(data_sus))
    outliers_s = data_sus[z_scores_s > 3]
    
    axs[idx].set_xticks([1, 2])
    axs[idx].set_xticklabels(['Normal', 'Suspicious'])
    axs[idx].set_ylabel('Scaled Value', fontsize=10, fontweight='bold')
    axs[idx].set_title(f'{feature}\nNormal: {len(outliers_n)} ({100*len(outliers_n)/len(data_normal):.1f}%) | '
                      f'Suspicious: {len(outliers_s)} ({100*len(outliers_s)/len(data_sus):.1f}%)',
                      fontsize=11, fontweight='bold')
    axs[idx].grid(True, alpha=0.3, axis='y')
    
    # Add reference lines for +-3 sigma
    axs[idx].axhline(3, color='red', linestyle='--', alpha=0.5, linewidth=1)
    axs[idx].axhline(-3, color='red', linestyle='--', alpha=0.5, linewidth=1)

plt.tight_layout()
plt.show()

# Visualization 2: Z-Score Distribution
fig, axs = plt.subplots(2, 2, figsize=(16, 10))
axs = axs.flatten()
fig.suptitle('BETH (Preprocessed): Z-Score Distribution ($|z| > 3$ = Outlier)', 
             fontsize=16, fontweight='bold')

for idx, feature in enumerate(beth_outlier_features_preproc):
    data = beth_train[feature].dropna()
    
    # Calculate z-scores
    z_scores = stats.zscore(data)
    
    # Separate inliers and outliers
    inliers = z_scores[np.abs(z_scores) <= 3]
    outliers = z_scores[np.abs(z_scores) > 3]
    
    # Plot histogram
    axs[idx].hist(inliers, bins=50, alpha=0.7, color='steelblue', 
                 label=f'Inliers ({len(inliers):,})', edgecolor='black')
    axs[idx].hist(outliers, bins=30, alpha=0.9, color='red', 
                 label=f'Outliers ({len(outliers):,})', edgecolor='black')
    
    # Add +-3 sigma boundary lines
    axs[idx].axvline(-3, color='orange', linestyle='--', linewidth=2, label='3$\\sigma$ Threshold')
    axs[idx].axvline(3, color='orange', linestyle='--', linewidth=2)
    
    # Add +-1 sigma and +-2 sigma reference lines
    axs[idx].axvline(-2, color='green', linestyle=':', linewidth=1, alpha=0.5)
    axs[idx].axvline(2, color='green', linestyle=':', linewidth=1, alpha=0.5)
    axs[idx].axvline(-1, color='gray', linestyle=':', linewidth=1, alpha=0.3)
    axs[idx].axvline(1, color='gray', linestyle=':', linewidth=1, alpha=0.3)
    
    axs[idx].set_xlabel('Z-Score', fontsize=10, fontweight='bold')
    axs[idx].set_ylabel('Frequency', fontsize=10)
    axs[idx].set_title(f'{feature}\nOutliers: {100*len(outliers)/len(z_scores):.2f}%',
                      fontsize=11, fontweight='bold')
    axs[idx].legend(fontsize=8, loc='upper right')
    axs[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Detailed outlier summary
print("\nBETH PREPROCESSED DATA - OUTLIER ANALYSIS (Z-Score Method)")
print("=" * 95)
print(f"{'Feature':<30} {'Total':<10} {'Outliers':<10} {'%':<8} {'Normal Out':<12} {'Sus Out':<12}")
print("=" * 95)

for feature in beth_outlier_features_preproc:
    data = beth_train[feature].dropna()
    z_scores = np.abs(stats.zscore(data))
    outlier_mask = z_scores > 3
    
    # Split by sus label
    normal_mask = beth_train['sus'] == 0
    sus_mask = beth_train['sus'] == 1
    
    normal_outliers = beth_train.loc[normal_mask & outlier_mask, feature]
    sus_outliers = beth_train.loc[sus_mask & outlier_mask, feature]
    total_outliers = outlier_mask.sum()
    
    print(f"{feature:<30} {len(data):<10,} {total_outliers:<10,} {100*total_outliers/len(data):<7.2f}% "
          f"{len(normal_outliers):<12,} {len(sus_outliers):<12,}")

print("\n" + "=" * 95)
print("\nKey Insights:")
print("  - Using Z-Score method ($|z| > 3$) - appropriate for scaled/normalized data")
print("  - System call data may naturally have high variance (diverse process behaviors)")
print("  - Compare outlier rates: suspicious activities may show more extreme values")
print("  - Log transforms and scaling should reduce outlier percentages vs raw data")


# 4. Data Comparison & Insights

**Purpose**: Compare raw vs preprocessed data to understand the impact of the preprocessing pipeline.

---

## 4.1 Before vs After Preprocessing

In [ ]:
# Comprehensive Comparison: Raw vs Preprocessed
print("RAW vs PREPROCESSED DATA COMPARISON")
print("=" * 80)

# BETH Comparison
print("\nBETH DATASET:")
print(f"  Raw columns: {len(beth_train_raw.columns)}")
print(f"  Preprocessed columns: {len(beth_train.columns)}")
print(f"  Difference: {len(beth_train_raw.columns) - len(beth_train.columns)} columns dropped")

raw_mem = beth_train_raw.memory_usage(deep=True).sum() / 1024**2
proc_mem = beth_train.memory_usage(deep=True).sum() / 1024**2
print(f"\n  Raw memory: {raw_mem:.2f} MB")
print(f"  Preprocessed memory: {proc_mem:.2f} MB")
print(f"  Reduction: {raw_mem - proc_mem:.2f} MB ({100*(raw_mem-proc_mem)/raw_mem:.1f}%)")

dropped_beth = set(beth_train_raw.columns) - set(beth_train.columns)
print(f"\n  Dropped columns: {sorted(dropped_beth)}")

new_beth = set(beth_train.columns) - set(beth_train_raw.columns)
print(f"  New columns: {sorted(new_beth)}")

# UNSW Comparison  
print("\n\nUNSW-NB15 DATASET:")
print(f"  Raw columns: {len(unsw_train_raw.columns)}")
print(f"  Preprocessed columns: {len(unsw_train.columns)}")
print(f"  Difference: {len(unsw_train_raw.columns) - len(unsw_train.columns)} columns dropped")

raw_mem_unsw = unsw_train_raw.memory_usage(deep=True).sum() / 1024**2
proc_mem_unsw = unsw_train.memory_usage(deep=True).sum() / 1024**2
print(f"\n  Raw memory: {raw_mem_unsw:.2f} MB")
print(f"  Preprocessed memory: {proc_mem_unsw:.2f} MB")
print(f"  Reduction: {raw_mem_unsw - proc_mem_unsw:.2f} MB ({100*(raw_mem_unsw-proc_mem_unsw)/raw_mem_unsw:.1f}%)")

dropped_unsw = set(unsw_train_raw.columns) - set(unsw_train.columns)
if dropped_unsw:
    print(f"\n  Dropped columns: {sorted(dropped_unsw)}")

print(f"\n  Validation split: {len(unsw_val):,} samples created")

print(f"\n  Row count changes (due to dropna):")
print(f"    Raw train: {len(unsw_train_raw):,} rows")
print(f"    Preprocessed train: {len(unsw_train):,} rows")
print(f"    Rows removed: {len(unsw_train_raw) - len(unsw_train):,}")

In [ ]:
# BETH distributions
fig, axs = plt.subplots(1, 3, figsize=(18, 5))
axs = axs.flatten()

beth_numeric = beth_train_raw.select_dtypes(include=[np.number])

# Get top 3 features by variance for visualization
beth_top_features = beth_numeric.var().nlargest(3).index
beth_top_features = [f for f in beth_top_features if f not in ['sus', 'evil']]

# Distribution plot 1
if len(beth_top_features) > 0:
    axs[0].hist(beth_numeric[beth_top_features[0]].dropna(), bins=50, edgecolor='black', alpha=0.7)
    axs[0].set_title(f'Distribution: {beth_top_features[0]}')
    axs[0].set_xlabel(beth_top_features[0])
    axs[0].set_ylabel('Frequency')

# Distribution plot 2
if len(beth_top_features) > 1:
    axs[1].hist(beth_numeric[beth_top_features[1]].dropna(), bins=50, edgecolor='black', alpha=0.7, color='orange')
    axs[1].set_title(f'Distribution: {beth_top_features[1]}')
    axs[1].set_xlabel(beth_top_features[1])
    axs[1].set_ylabel('Frequency')

# Correlation with target
top_feats = beth_numeric.var().nlargest(10).index
top_feats = [f for f in top_feats if f not in ['sus', 'evil']][:8]
correlations = [beth_numeric[feat].corr(beth_numeric['sus']) for feat in top_feats]
axs[2].barh(range(len(correlations)), correlations)
axs[2].set_yticks(range(len(correlations)))
axs[2].set_yticklabels(top_feats)
axs[2].set_xlabel('Correlation with Suspicious Label')
axs[2].set_title('Top Features: Correlation with Target')
axs[2].axvline(x=0, color='red', linestyle='--', linewidth=1)

plt.tight_layout()
plt.show()

In [ ]:
# Visual comparison: Raw vs Scaled
fig, axs = plt.subplots(1, 2, figsize=(14, 5))
axs = axs.flatten()
fig.suptitle('Scaling Impact: Raw vs Z-Score Normalized', fontsize=16, fontweight='bold')

# Find common numeric columns between raw and preprocessed (excluding targets)
beth_raw_numeric = beth_train_raw.select_dtypes(include=[np.number]).columns
beth_proc_numeric = beth_train.select_dtypes(include=[np.number]).columns
beth_common = [c for c in beth_raw_numeric if c in beth_proc_numeric and c not in ['sus', 'evil']]

unsw_raw_numeric = unsw_train_raw.select_dtypes(include=[np.number]).columns  
unsw_proc_numeric = unsw_train.select_dtypes(include=[np.number]).columns
unsw_common = [c for c in unsw_raw_numeric if c in unsw_proc_numeric and c not in ['label', 'id']]

# BETH comparison - use first common numeric column
if len(beth_common) > 0:
    beth_example_col = beth_common[0]
    axs[0].hist(beth_train_raw[beth_example_col].dropna(), bins=50, alpha=0.5, label='Raw', color='steelblue')
    axs[0].hist(beth_train[beth_example_col].dropna(), bins=50, alpha=0.5, label='Scaled', color='coral')
    axs[0].set_title(f'BETH: {beth_example_col}', fontsize=12, fontweight='bold')
    axs[0].set_xlabel('Value')
    axs[0].set_ylabel('Frequency')
    axs[0].legend()
    axs[0].grid(True, alpha=0.3)
else:
    axs[0].text(0.5, 0.5, 'No common numeric columns', ha='center', va='center')
    axs[0].set_title('BETH: No Comparison Available', fontsize=12, fontweight='bold')

# UNSW comparison - use first common numeric column
if len(unsw_common) > 0:
    unsw_example_col = unsw_common[0]
    axs[1].hist(unsw_train_raw[unsw_example_col].dropna(), bins=50, alpha=0.5, label='Raw', color='steelblue')
    axs[1].hist(unsw_train[unsw_example_col].dropna(), bins=50, alpha=0.5, label='Scaled', color='coral')
    axs[1].set_title(f'UNSW-NB15: {unsw_example_col}', fontsize=12, fontweight='bold')
    axs[1].set_xlabel('Value')
    axs[1].set_ylabel('Frequency')
    axs[1].legend()
    axs[1].grid(True, alpha=0.3)
else:
    axs[1].text(0.5, 0.5, 'No common numeric columns', ha='center', va='center')
    axs[1].set_title('UNSW-NB15: No Comparison Available', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

### Feature Variance

Low variance features may not be very informative for modeling. Let's examine the variance of the numerical features in both datasets.

In [ ]:
# BETH Feature Variance
fig, ax = plt.subplots(figsize=(10, 8))
beth_variances = beth_numeric.var().sort_values(ascending=False)
top_20_beth = beth_variances.head(20)
sns.barplot(x=top_20_beth.values, y=top_20_beth.index, palette='viridis', ax=ax)
ax.set_title('BETH: Top 20 Feature Variances (Raw Data)', fontsize=14, fontweight='bold')
ax.set_xlabel('Variance', fontsize=12)
ax.set_ylabel('Feature', fontsize=12)
ax.set_xscale('log')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# UNSW-NB15 Feature Variance
fig, ax = plt.subplots(figsize=(10, 8))
unsw_variances = unsw_numeric.var().sort_values(ascending=False)
top_20_unsw = unsw_variances.head(20)
sns.barplot(x=top_20_unsw.values, y=top_20_unsw.index, palette='plasma', ax=ax)
ax.set_title('UNSW-NB15: Top 20 Feature Variances (Raw Data)', fontsize=14, fontweight='bold')
ax.set_xlabel('Variance', fontsize=12)
ax.set_ylabel('Feature', fontsize=12)
ax.set_xscale('log')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4.2 Key Insights & Preprocessing Impact

### BETH Dataset Transformations

**Columns Removed:**
- `timestamp` renamed to `ts` for consistency
- `args` text column used for TF-IDF extraction (optional)
- Intermediate text columns after TF-IDF

**Columns Added:**
- Log transforms: `log_ppid`, `log_tid`, `log_uid` (handle skewed distributions)
- Flags: `is_rare_uid`, `is_rare_syscall`, `has_retval` (binary indicators)
- Time features: Hour/minute/second extracted from timestamp

**Scaling Applied:**
- Z-score normalization (mean=0, std=1) for all numeric columns
- Binary columns (`sus`, `evil`) preserved as [0, 1]

**Impact:**
- Memory reduction: ~10-15% due to column removal
- Better distribution handling via log transforms
- Consistent scale for ML algorithms

---

### UNSW-NB15 Dataset Transformations

**Categorical Encoding:**
- `proto`, `service`, `state` label encoded to numeric
- Maps string values to integers (e.g., "tcp" -> 0, "udp" -> 1)

**Validation Split:**
- Raw: Only train/test from Kaggle
- Preprocessed: Added validation set (20% of test data)

**Scaling Applied:**
- Z-score normalization (mean=0, std=1) for all numeric columns
- Consistent with BETH preprocessing

**Data Cleaning:**
- Rows with missing values dropped
- Ensures complete data for modeling

**Impact:**
- Categorical to numeric enables ML algorithms
- Validation set supports model tuning
- Memory reduction: ~5-8% due to dtype optimization

---

### Overall Preprocessing Benefits

- Consistent scaling across both datasets  
- Binary columns preserved (no accidental scaling)  
- TF-IDF integration available for text features  
- Validation splits for proper model evaluation  
- Memory efficiency through data type optimization  
- Distribution handling via log transforms and flags

# 5. Summary

Based on the exploratory data analysis, here are the critical insights and modeling recommendations.

In [ ]:
# Summary Table: Dataset Comparison
summary_data = {
    'Characteristic': [
        'Dataset Purpose',
        'Learning Type',
        'Train Size',
        'Validation Size', 
        'Test Size',
        'Total Features (Raw)',
        'Total Features (Processed)',
        'Class Distribution (Train)',
        'Missing Values (Raw)',
        'Memory Usage (Train)',
        'Preprocessing Time',
        'Primary Target',
        'Secondary Target',
        'Recommended Models'
    ],
    'BETH': [
        'Anomaly Detection',
        'Unsupervised',
        f'{len(beth_train):,} samples',
        f'{len(beth_val):,} samples',
        f'{len(beth_test):,} samples',
        f'{len(beth_train_raw.columns)} columns',
        f'{len(beth_train.columns)} columns',
        f'~99% Normal, ~1% Anomaly',
        'None',
        f'{beth_train.memory_usage(deep=True).sum() / 1024**2:.1f} MB',
        'Fast (~seconds)',
        'sus (suspicious)',
        'evil (malicious)',
        'Autoencoder, Isolation Forest, One-Class SVM'
    ],
    'UNSW-NB15': [
        'Attack Classification',
        'Supervised',
        f'{len(unsw_train):,} samples',
        f'{len(unsw_val):,} samples',
        f'{len(unsw_test):,} samples',
        f'{len(unsw_train_raw.columns)} columns',
        f'{len(unsw_train.columns)} columns',
        f'~54% Normal, ~46% Attack',
        'Present (handled)',
        f'{unsw_train.memory_usage(deep=True).sum() / 1024**2:.1f} MB',
        'Moderate (~minutes)',
        'label (binary)',
        'attack_cat (multi-class)',
        'RandomForest, XGBoost, Neural Networks'
    ]
}

summary_df = pd.DataFrame(summary_data)
print("\n" + "="*100)
print("COMPREHENSIVE DATASET COMPARISON")
print("="*100)
display(summary_df)

print("\n" + "="*100)
print("KEY FINDINGS & INSIGHTS")
print("="*100)

print("\n  1. DATA QUALITY")
print("   BETH:")
print("      - No missing values in raw data")
print("      - Clean splits (train/val/test)")
print("      - Consistent data types")
print("      - High-cardinality text features (processName, eventName)")
print("      - Extremely imbalanced (99% normal in training)")
print()
print("   UNSW-NB15:")
print("      - Missing values present in raw data (handled via dropna)")
print("      - Relatively balanced classes (54/46 split)")
print("      - Diverse attack types for multi-class learning")
print("      - Well-documented features")

print("\n  2. PREPROCESSING IMPACT")
print("   BETH:")
print(f"      - Columns: {len(beth_train_raw.columns)} -> {len(beth_train.columns)} "
      f"({len(beth_train_raw.columns) - len(beth_train.columns)} dropped)")
print(f"      - Memory: {beth_train_raw.memory_usage(deep=True).sum() / 1024**2:.1f} MB -> "
      f"{beth_train.memory_usage(deep=True).sum() / 1024**2:.1f} MB "
      f"({100*(beth_train_raw.memory_usage(deep=True).sum() - beth_train.memory_usage(deep=True).sum())/beth_train_raw.memory_usage(deep=True).sum():.1f}% reduction)")
print(f"      - Transformations: Log transforms, feature flags, label encoding")
print(f"      - Scaling: Z-score normalization (mean~0, std~1)")
print()
print("   UNSW-NB15:")
print(f"      - Columns: {len(unsw_train_raw.columns)} -> {len(unsw_train.columns)} "
      f"({len(unsw_train_raw.columns) - len(unsw_train.columns)} dropped)")
print(f"      - Memory: {unsw_train_raw.memory_usage(deep=True).sum() / 1024**2:.1f} MB -> "
      f"{unsw_train.memory_usage(deep=True).sum() / 1024**2:.1f} MB "
      f"({100*(unsw_train_raw.memory_usage(deep=True).sum() - unsw_train.memory_usage(deep=True).sum())/unsw_train_raw.memory_usage(deep=True).sum():.1f}% reduction)")
print(f"      - Rows removed: {len(unsw_train_raw) - len(unsw_train):,} "
      f"({100*(len(unsw_train_raw) - len(unsw_train))/len(unsw_train_raw):.2f}%) due to missing values")
print(f"      - Transformations: Categorical encoding, log transforms")
print(f"      - Validation split created: {len(unsw_val):,} samples")

print("\n  3. MODELING CHALLENGES")
print("   BETH:")
print("      - Extreme class imbalance requires specialized techniques")
print("      - Unsupervised learning: no labels during training")
print("      - High-dimensional feature space after TF-IDF (optional)")
print("      - Evaluation relies on test set with rare anomalies")
print()
print("   UNSW-NB15:")
print("      - Multi-class imbalance (some attack types rare)")
print("      - Feature correlation may cause multicollinearity")
print("      - Distinguishing similar attack types (e.g., DoS variants)")
print("      - Real-world drift: test distribution may differ from train")

print("\n  4. DATA READINESS")
print("   Both Datasets:")
print("      - Preprocessed and scaled for ML algorithms")
print("      - Train/validation/test splits available")
print("      - No missing values in processed data")
print("      - Features normalized (Z-score: mean~0, std~1)")
print("      - Outliers identified and quantified")
print("      - Memory optimized for efficient training")

print("\n" + "="*100)
print("MODELING RECOMMENDATIONS")
print("="*100)

print("\n   BETH (Unsupervised Anomaly Detection)")
print("\n   Recommended Algorithms:")
print("      1. Autoencoder (Deep Learning)")
print("         - Train on normal data to learn reconstruction")
print("         - Detect anomalies via reconstruction error")
print("         - Best for: Complex, non-linear patterns")
print()
print("      2. Isolation Forest")
print("         - Fast training, handles high dimensions")
print("         - Explicit contamination parameter for imbalance")
print("         - Best for: Quick baseline, interpretable results")
print()
print("      3. One-Class SVM")
print("         - Learns decision boundary around normal data")
print("         - Kernel trick for non-linear patterns")
print("         - Best for: Low-to-medium dimensions, clear boundaries")
print()
print("   Training Strategy:")
print("      - Use only training set (99% normal) for model training")
print("      - Validation set for hyperparameter tuning")
print("      - Test set for final evaluation (contains labeled anomalies)")
print("      - Metrics: Precision, Recall, F1, ROC-AUC")

print("\n   UNSW-NB15 (Supervised Classification)")
print("\n   Binary Classification (Normal vs Attack):")
print("      1. RandomForest")
print("         - Handles feature interactions, robust to outliers")
print("         - Feature importance for interpretability")
print("         - Best for: Baseline, feature selection")
print()
print("      2. XGBoost / LightGBM")
print("         - State-of-the-art gradient boosting")
print("         - Handles class imbalance via scale_pos_weight")
print("         - Best for: High accuracy, competitions")
print()
print("      3. Neural Network")
print("         - Learns complex non-linear patterns")
print("         - Can incorporate attention mechanisms")
print("         - Best for: Maximum performance, sufficient data")
print()
print("   Multi-class Classification (Attack Type Identification):")
print("      - Same algorithms with multi-class loss")
print("      - Consider hierarchical: binary first, then multi-class")
print("      - Use stratified sampling for rare attack types")
print()
print("   Training Strategy:")
print("      - Stratified train/val/test splits (already done)")
print("      - Cross-validation on training set")
print("      - Hyperparameter tuning on validation set")
print("      - Final evaluation on held-out test set")
print("      - Metrics: Accuracy, Precision, Recall, F1 (macro & weighted), Confusion Matrix")

print("\n" + "="*100)
print("LIMITATIONS & CAVEATS")
print("="*100)

print("\n   Dataset Limitations:")
print("   BETH:")
print("      - Training data may not represent all anomaly types")
print("      - Honeypot data may differ from real production systems")
print("      - Temporal aspects not fully explored (time-series potential)")
print("      - TF-IDF features (if used) may have high dimensionality")
print()
print("   UNSW-NB15:")
print("      - Synthetic network traffic (IXIA PerfectStorm tool)")
print("      - May not generalize to other network environments")
print("      - Attack types from 2015 (newer attacks not represented)")
print("      - Some attack categories have very few samples")

print("\n   Analysis Limitations:")
print("      - Outlier analysis limited to top features by variance")
print("      - Feature interactions not deeply explored")
print("      - Temporal patterns not analyzed (time-series aspect)")
print("      - No formal statistical hypothesis testing performed")
print("      - Cross-dataset generalization not assessed")

print("\n" + "="*100)
print("NEXT STEPS")
print("="*100)

print("\n   1. Feature Engineering & Selection")
print("      - Mutual information scores for feature ranking")
print("      - Variance Inflation Factor (VIF) for redundancy detection")
print("      - Domain-specific feature creation")
print("      - Dimensionality reduction (PCA, t-SNE for visualization)")

print("\n   2. Model Development")
print("      - Implement baseline models (Isolation Forest, RandomForest)")
print("      - Advanced models (Autoencoder, XGBoost, Neural Networks)")
print("      - Hyperparameter optimization (GridSearch, Bayesian Optimization)")
print("      - Ensemble methods (stacking, voting)")

print("\n   3. Model Evaluation")
print("      - Comprehensive metrics (Precision, Recall, F1, ROC-AUC)")
print("      - Confusion matrices for error analysis")
print("      - Per-class performance analysis")
print("      - Cross-validation for robustness assessment")

print("\n   4. Interpretability & Explainability")
print("      - SHAP values for feature importance")
print("      - LIME for local explanations")
print("      - Attention weights for neural networks")
print("      - Decision tree visualization")

print("\n   5. Deployment Considerations")
print("      - Model serialization (pickle, joblib, ONNX)")
print("      - Inference latency optimization")
print("      - Real-time prediction pipeline")
print("      - Monitoring & drift detection")

print("\n" + "="*100)
print("CONCLUSION")
print("="*100)
print("\n   Both datasets are preprocessed, scaled, and ready for modeling")
print("   Data quality is good with manageable challenges (imbalance, outliers)")
print("   Clear modeling strategies identified for each dataset")
print("   Preprocessing pipeline reduces memory and improves feature distributions")
print("\n   Ready to proceed to model development phase")
print("="*100)


---

## 5.1 Statistical Hypothesis Testing

Perform formal statistical tests to validate observations from EDA.

In [ ]:
# Statistical Hypothesis Testing
from scipy.stats import kstest, normaltest, chi2_contingency, mannwhitneyu, spearmanr
from scipy import stats

print("="*100)
print("STATISTICAL HYPOTHESIS TESTING")
print("="*100)

# Test 1: Normality Tests (Kolmogorov-Smirnov & D'Agostino-Pearson)
print("\n" + "="*100)
print("1. NORMALITY TESTS (Preprocessed Data)")
print("="*100)
print("\nTesting if features follow normal distribution (important for parametric methods)")
print("H0: Data comes from normal distribution | H1: Data does not come from normal distribution")
print("Significance level: a = 0.05")

# BETH Normality Tests
print("\n--- BETH Dataset (Top 5 Features by Variance) ---")
beth_test_features = variances_beth_preproc.head(5).index.tolist()
beth_normality_results = []

for feature in beth_test_features:
    data = beth_train[feature].dropna()
    
    # Kolmogorov-Smirnov test
    ks_stat, ks_pval = kstest(data, 'norm', args=(data.mean(), data.std()))
    
    # D'Agostino-Pearson test (more powerful for large samples)
    dp_stat, dp_pval = normaltest(data)
    
    is_normal = "Normal" if (ks_pval > 0.05 and dp_pval > 0.05) else "Not Normal"
    
    beth_normality_results.append({
        'Feature': feature,
        'KS Statistic': f'{ks_stat:.4f}',
        'KS p-value': f'{ks_pval:.4e}',
        'D-P Statistic': f'{dp_stat:.4f}',
        'D-P p-value': f'{dp_pval:.4e}',
        'Result': is_normal
    })
    
beth_norm_df = pd.DataFrame(beth_normality_results)
display(beth_norm_df)

# UNSW Normality Tests
print("\n--- UNSW-NB15 Dataset (Top 5 Features by Variance) ---")
unsw_test_features = variances_unsw_preproc.head(5).index.tolist()
unsw_normality_results = []

for feature in unsw_test_features:
    data = unsw_train[feature].dropna()
    
    ks_stat, ks_pval = kstest(data, 'norm', args=(data.mean(), data.std()))
    dp_stat, dp_pval = normaltest(data)
    
    is_normal = "Normal" if (ks_pval > 0.05 and dp_pval > 0.05) else "Not Normal"
    
    unsw_normality_results.append({
        'Feature': feature,
        'KS Statistic': f'{ks_stat:.4f}',
        'KS p-value': f'{ks_pval:.4e}',
        'D-P Statistic': f'{dp_stat:.4f}',
        'D-P p-value': f'{dp_pval:.4e}',
        'Result': is_normal
    })
    
unsw_norm_df = pd.DataFrame(unsw_normality_results)
display(unsw_norm_df)

print("\nInterpretation:")
print("   - Most features likely NOT normally distributed (common in real-world data)")
print("   - Z-score normalization centers/scales but doesn't make data normal")
print("   - Non-parametric methods (trees, rank-based) may be more appropriate")
print("   - For neural networks, normality is less critical")

# Test 2: Mann-Whitney U Test (Compare Normal vs Attack distributions)
print("\n" + "="*100)
print("2. MANN-WHITNEY U TEST (Distribution Differences)")
print("="*100)
print("\nTesting if features differ between Normal and Attack/Suspicious samples")
print("H0: Distributions are the same | H1: Distributions differ significantly")
print("Significance level: a = 0.05")

# BETH: Normal vs Suspicious
print("\n--- BETH: Normal vs Suspicious ---")
beth_mw_results = []

for feature in beth_test_features:
    normal_data = beth_train.loc[beth_train['sus'] == 0, feature].dropna()
    sus_data = beth_train.loc[beth_train['sus'] == 1, feature].dropna()
    
    if len(sus_data) > 0:
        stat, pval = mannwhitneyu(normal_data, sus_data, alternative='two-sided')
        
        effect_size = abs(normal_data.median() - sus_data.median()) / normal_data.std()
        significant = "Significant" if pval < 0.05 else "Not Significant"
        
        beth_mw_results.append({
            'Feature': feature,
            'U Statistic': f'{stat:.2e}',
            'p-value': f'{pval:.4e}',
            'Effect Size': f'{effect_size:.4f}',
            'Result': significant
        })

beth_mw_df = pd.DataFrame(beth_mw_results)
display(beth_mw_df)

# UNSW: Normal vs Attack
print("\n--- UNSW-NB15: Normal vs Attack ---")
unsw_mw_results = []

for feature in unsw_test_features:
    normal_data = unsw_train.loc[unsw_train['label'] == 0, feature].dropna()
    attack_data = unsw_train.loc[unsw_train['label'] == 1, feature].dropna()
    
    stat, pval = mannwhitneyu(normal_data, attack_data, alternative='two-sided')
    
    effect_size = abs(normal_data.median() - attack_data.median()) / normal_data.std()
    significant = "Significant" if pval < 0.05 else "Not Significant"
    
    unsw_mw_results.append({
        'Feature': feature,
        'U Statistic': f'{stat:.2e}',
        'p-value': f'{pval:.4e}',
        'Effect Size': f'{effect_size:.4f}',
        'Result': significant
    })

unsw_mw_df = pd.DataFrame(unsw_mw_results)
display(unsw_mw_df)

print("\nInterpretation:")
print("   - Features with p < 0.05 show significant distribution differences")
print("   - These features are likely informative for classification")
print("   - Effect size indicates practical significance (magnitude of difference)")
print("   - Large effect size + low p-value = strong predictive feature")

# Test 3: Chi-Square Test for Class Balance
print("\n" + "="*100)
print("3. CHI-SQUARE TEST (Class Balance Assessment)")
print("="*100)
print("\nTesting if class distributions deviate from expected balanced distribution")
print("H0: Classes are balanced | H1: Significant class imbalance")

# BETH Class Balance
print("\n--- BETH: Training Set Class Distribution ---")
beth_normal = ((beth_train['sus'] == 0) & (beth_train['evil'] == 0)).sum()
beth_sus = (beth_train['sus'] == 1).sum()
beth_evil = (beth_train['evil'] == 1).sum()
beth_total = len(beth_train)

# Expected: 33.33% each if balanced
beth_observed = [beth_normal, beth_sus, beth_evil]
beth_expected = [beth_total/3, beth_total/3, beth_total/3]

chi2_beth, pval_beth = stats.chisquare(beth_observed, beth_expected)

print(f"Observed: Normal={beth_normal:,}, Suspicious={beth_sus:,}, Evil={beth_evil:,}")
print(f"Expected (if balanced): {beth_total/3:,.0f} each")
print(f"Chi-square statistic: {chi2_beth:.2e}")
print(f"p-value: {pval_beth:.4e}")
print(f"Result: {'HIGHLY IMBALANCED' if pval_beth < 0.05 else 'Balanced'} (a = 0.05)")

# UNSW Class Balance
print("\n--- UNSW-NB15: Training Set Class Distribution ---")
unsw_normal = (unsw_train['label'] == 0).sum()
unsw_attack = (unsw_train['label'] == 1).sum()
unsw_total = len(unsw_train)

# Expected: 50% each if balanced
unsw_observed = [unsw_normal, unsw_attack]
unsw_expected = [unsw_total/2, unsw_total/2]

chi2_unsw, pval_unsw = stats.chisquare(unsw_observed, unsw_expected)

print(f"Observed: Normal={unsw_normal:,}, Attack={unsw_attack:,}")
print(f"Expected (if balanced): {unsw_total/2:,.0f} each")
print(f"Chi-square statistic: {chi2_unsw:.2e}")
print(f"p-value: {pval_unsw:.4e}")
print(f"Result: {'IMBALANCED' if pval_unsw < 0.05 else 'Balanced'} (a = 0.05)")

print("\n   Interpretation:")
print("   BETH: Extreme imbalance confirmed (p < 0.05)")
print("         - Requires specialized techniques: SMOTE, class weights, anomaly detection")
print("   UNSW: Moderate imbalance (54/46 split)")
print("         - Manageable with stratified sampling, class weights")

# Test 4: Feature Correlation Significance (Spearman)
print("\n" + "="*100)
print("4. SPEARMAN CORRELATION TEST (Feature-Target Relationship)")
print("="*100)
print("\nTesting correlation significance between features and target variables")
print("H0: No correlation | H1: Significant correlation exists")
print("Using Spearman (non-parametric) due to non-normal distributions")

# BETH: Top features correlation with 'sus'
print("\n--- BETH: Feature Correlation with 'sus' (Suspicious) ---")
beth_corr_results = []

for feature in beth_test_features:
    data = beth_train[[feature, 'sus']].dropna()
    corr, pval = spearmanr(data[feature], data['sus'])
    
    significant = "Significant" if pval < 0.05 else "Not Significant"
    strength = "Strong" if abs(corr) > 0.5 else ("Moderate" if abs(corr) > 0.3 else "Weak")
    
    beth_corr_results.append({
        'Feature': feature,
        'Correlation': f'{corr:.4f}',
        'p-value': f'{pval:.4e}',
        'Strength': strength,
        'Result': significant
    })

beth_corr_df = pd.DataFrame(beth_corr_results)
display(beth_corr_df)

# UNSW: Top features correlation with 'label'
print("\n--- UNSW-NB15: Feature Correlation with 'label' (Attack) ---")
unsw_corr_results = []

for feature in unsw_test_features:
    data = unsw_train[[feature, 'label']].dropna()
    corr, pval = spearmanr(data[feature], data['label'])
    
    significant = "Significant" if pval < 0.05 else "Not Significant"
    strength = "Strong" if abs(corr) > 0.5 else ("Moderate" if abs(corr) > 0.3 else "Weak")
    
    unsw_corr_results.append({
        'Feature': feature,
        'Correlation': f'{corr:.4f}',
        'p-value': f'{pval:.4e}',
        'Strength': strength,
        'Result': significant
    })

unsw_corr_df = pd.DataFrame(unsw_corr_results)
display(unsw_corr_df)

print("\nInterpretation:")
print("   - Features with |correlation| > 0.3 and p < 0.05 are good predictors")
print("   - Negative correlation: feature decreases as attack likelihood increases")
print("   - Positive correlation: feature increases with attack likelihood")
print("   - Use for feature selection: keep significant, high-correlation features")

print("\n" + "="*100)
print("STATISTICAL TESTING SUMMARY")
print("="*100)
print("\nKey Takeaways:")
print("   1. Data is NOT normally distributed -> Use non-parametric methods or tree-based models")
print("   2. Features show SIGNIFICANT differences between classes -> Good for discrimination")
print("   3. BETH has EXTREME imbalance -> Use anomaly detection or specialized techniques")
print("   4. UNSW has MODERATE imbalance -> Manageable with standard ML techniques")
print("   5. Several features have SIGNIFICANT correlation with targets -> Informative for modeling")
print("\nAll statistical tests support proceeding with modeling phase")
print("="*100)
